# Projeto 2 - Ciência dos Dados

Nome: Pedro Braga Carani

Nome: Gustavo

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [3]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [4]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [6]:
#Produto escolhido:
produto = 'fone jbl'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [8]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

Nesse projeto o tweet é classificado como relevante quando ele possui uma crítica ao produto.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [9]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;#/,\n https]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [10]:
fone_Treinamento=pd.read_excel("fone jbl.xlsx", sheet_name="Treinamento")
fone_Treinamento


,Treinamento,classificacao
0,eu tenho um fone bluetooth da jbl e toda vez q...,1
1,"saudade do meu fone jbl que paguei 80 reis, nã...",1
2,tinha uma menina do meu lado vendo mv de kpop ...,1
3,odeio a porra do fone da jbl vsf,1
4,"velho esse fone da #jbl é fora de série, qual...",1
5,fone de ouvido supra auricular com microfone j...,0
6,fone de ouvido jbl t110 bluetooth\nfrequência ...,0
7,meu irmao querendo comprar meu fone da jbl htt...,1
8,"@corujosilva o meu fone não foi caro, é o mais...",1
9,acho que a melhor coisa que eu já comprei foi ...,1


In [11]:
#fone

In [12]:
fone_r=fone_Treinamento[fone_Treinamento["classificacao"]==1]
textor="".join(fone_r["Treinamento"])
texto_cleanr=cleanup(textor)
lista_r=texto_cleanr.split()
freq_r=pd.Series(lista_r).value_counts(True)
#freq_r

fone_ir=fone_Treinamento[fone_Treinamento["classificacao"]==0]
textoi="".join(fone_r["Treinamento"])
texto_cleani=cleanup(textoi)
lista_ir=texto_cleani.split()
freq_ir=pd.Series(lista_ir).value_counts(True)
#freq_ir


# Classificador Naive-Bayes

In [13]:
#Dados empiricos
prelevante=0.69
pirrelevante=0.31



tweet=fone_Treinamento["Treinamento"][10]
frase=cleanup(tweet)
frase1=frase.split()

#P(Frase|Relevante)
Pfraserelevante=freq_r.loc[frase1].prod()
#P(Frase|Irrelevante)
Pfraseirrelevante=freq_ir.loc[frase1].prod()
    
#P(Relevante|Frase)    
Prelevantefrase=Pfraserelevante*prelevante
                    
#P(Irrelevante|Frase)
Pirrelevantefrase=Pfraseirrelevante*pirrelevante

if Prelevantefrase > Pirrelevantefrase:
    print("Relevante")
    
else:
    print ("Irrelevante")

    
print (Prelevantefrase)
print (Pirrelevantefrase)
                    

Relevante
1.0735932556936017e-37
4.823389989348066e-38


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if sys.path[0] == '':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [14]:
fone_teste=pd.read_excel("fone jbl.xlsx", sheet_name="Teste")

relevante=0
irrelevante=0
for i in fone_teste["Teste"]:
    tweet=fone_teste["Teste"][i]
    frase=cleanup(tweet)
    frase1=frase.split()
    
#P(Frase|Relevante)
    Pfraserelevante=freq_r.loc[frase1].prod()
#P(Frase|Irrelevante)
    Pfraseirrelevante=freq_ir.loc[frase1].prod()
    
#P(Relevante|Frase)    
    Prelevantefrase=Pfraserelevante*prelevante
                    
#P(Irrelevante|Frase)
    Pirrelevantefrase=Pfraseirrelevante*pirrelevante

    if Prelevantefrase > Pirrelevantefrase:
        #print("Relevante")
        relevante+=1
    
    else:
        print ("Irrelevante")
        irrelevante+=1



KeyError: 'eu nao tenho sorte com fone cara, nao deu 1 mes que comprei esse da jbl, paguei caro e ja ta bugado'

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**